In [ ]:
pip install bayesian-optimization

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for bayesian-optimization: filename=bayesian_optimization-1.2.0-py3-none-any.whl size=11685 sha256=63343ddce003563a7a46b9df4c43d6de1f3943cb5cba65df17279224f5a76873
  Stored in directory: /root/.cache/pip/wheels/fd/9b/71/f127d694e02eb40bcf18c7ae9613b88a6be4470f57a8528c5b
Successfully built bayesian-optimization


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import metrics
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer
import tensorflow as tf
from keras import regularizers
from sklearn.preprocessing import MinMaxScaler
from bayes_opt import BayesianOptimization

In [ ]:
kt_csv = pd.read_csv('/content/drive/MyDrive/kt/round1_train.csv')

In [ ]:
kt_csv

In [ ]:
kt_csv.isnull().sum()

In [ ]:
# mfreq1 = kt_csv['광고 응답 광고주 도메인'].value_counts().idxmax()
# mfreq2 = kt_csv['국가코드 ID'].value_counts().idxmax()
# mfreq3 = kt_csv['ADID'].value_counts().idxmax()
mfreq4 = kt_csv['WUID (웹 유저 ID)'].value_counts().idxmax()
# mfreq5 = kt_csv['광고 응답 소재 카테고리'].value_counts().idxmax()
# mfreq6 = kt_csv['OS 버전 ID'].value_counts().idxmax()

In [ ]:
# kt_csv['광고 응답 광고주 도메인'] = kt_csv['광고 응답 광고주 도메인'].fillna(mfreq1)
# kt_csv['국가코드 ID'] = kt_csv['국가코드 ID'].fillna(mfreq2)
# kt_csv['ADID'] = kt_csv['ADID'].fillna(mfreq3)
kt_csv['WUID (웹 유저 ID)'] = kt_csv['WUID (웹 유저 ID)'].fillna(mfreq4)
# kt_csv['광고 응답 소재 카테고리'] = kt_csv['광고 응답 소재 카테고리'].fillna(mfreq5)
# kt_csv['OS 버전 ID'] = kt_csv['OS 버전 ID'].fillna(mfreq6)

In [ ]:
kt_csv.corr()

In [ ]:
kt_csv.isnull().sum()

In [ ]:
kt_csv.info()

In [ ]:
# kt_csv['시각'] = kt_csv['시각'].astype('category')
# kt_csv['ADID'] = kt_csv['ADID'].astype('category')
# kt_csv['DSP ID'] = kt_csv['DSP ID'].astype('category')
# kt_csv['노출 ID'] = kt_csv['노출 ID'].astype('category')
# kt_csv['SSP 입찰ID'] = kt_csv['SSP 입찰ID'].astype('category')
# kt_csv['DSP 입찰ID'] = kt_csv['DSP 입찰ID'].astype('category')
# kt_csv['AX 낙찰ID'] = kt_csv['AX 낙찰ID'].astype('category')
kt_csv['WUID (웹 유저 ID)'] = kt_csv['WUID (웹 유저 ID)'].astype('category')
# kt_csv['광고 응답 소재 카테고리'] = kt_csv['AX 낙찰ID'].astype('category')
# kt_csv['매체 ID'] = kt_csv['매체 ID'].astype('category')
# kt_csv['애드유닛 ID'] = kt_csv['애드유닛 ID'].astype('category')
# kt_csv['광고 응답 광고주 도메인'] = kt_csv['광고 응답 광고주 도메인'].astype('category')

In [ ]:
ndf = kt_csv.drop(['시각', '환율', 'ADID', 'ADID 타입', '매체 ID', 'OS 종류', '노출 ID', 'SSP 입찰ID', 'DSP 입찰ID', 'AX 낙찰ID', '광고 응답 소재 카테고리', '광고 응답 광고주 도메인', '국가코드 ID', 'OS 버전 ID', 'P2','P5'], axis = 'columns')

In [ ]:
labelencoder=LabelEncoder()
for col in ndf.columns:
    ndf[col] = labelencoder.fit_transform(ndf[col])

In [ ]:
ny = ndf['winning'].values
nx = ndf.drop(['winning'], axis=1).values
nx = (nx - nx.min()) / (nx.max() - nx.min())

In [ ]:
nny = ndf['winning'].values

In [ ]:
ny = tf.keras.utils.to_categorical(ny, num_classes=2) #one-hot encoding

In [ ]:
nx.shape, ny.shape

((8525512, 8), (8525512, 2))

In [ ]:
np.isnan(nx).any(), np.isnan(ny).any()

(False, False)

In [ ]:
nx

array([[6.37857479e-06, 2.95540632e-04, 0.00000000e+00, ...,
        5.26232420e-04, 2.12619160e-06, 1.09047052e-01],
       [5.31547899e-06, 6.74002736e-04, 0.00000000e+00, ...,
        9.45424914e-01, 2.12619160e-06, 9.45247377e-01],
       [5.31547899e-06, 2.62584662e-04, 2.12619160e-06, ...,
        8.13774320e-01, 4.08228787e-04, 8.13609540e-01],
       ...,
       [5.31547899e-06, 6.07027701e-04, 2.12619160e-06, ...,
        8.76035588e-01, 9.42965973e-04, 8.75872935e-01],
       [5.31547899e-06, 9.12136195e-04, 0.00000000e+00, ...,
        4.44099765e-01, 5.31547899e-06, 4.43936048e-01],
       [5.31547899e-06, 9.12136195e-04, 0.00000000e+00, ...,
        4.44099765e-01, 5.31547899e-06, 4.43936048e-01]])

In [ ]:
from sklearn.model_selection import train_test_split

x_train, x_val, y_train, y_val = train_test_split(nx, ny, test_size = 0.1, random_state=42)

In [ ]:
xx_train, xx_val, yy_train, yy_val = train_test_split(nx, nny, test_size = 0.1, random_state=42)

In [ ]:
# #XGBoost모델의 정확도
# from xgboost import XGBClassifier

# xgbm = XGBClassifier(
#                      colsample_bytree = 0.5373638966605134,
#                      gamma = 96.95379387360022,
#                      max_depth = 9,
#                      min_child_weight = 2.860639159783224,
#                      subsample = 0.6333975735601048,
#                      tree_method='gpu_hist', 
#                      n_estimators=8940, 
#                      learning_rate=0.1388146251654753
#                     )
# xgbm.fit(xx_train, yy_train)
# xgbm.score(xx_val, yy_val)

In [ ]:
# # 의사결정트리모델의 정확도
# from sklearn import tree

# treem = tree.DecisionTreeClassifier(
#                                     max_depth=None, 
#                                     min_samples_split=2, 
#                                     max_features=None, 
#                                     min_samples_leaf=1,
#                                     criterion = 'entropy'
#                                     )
# treem.fit(x_train, y_train)
# treem.score(x_val, y_val)

In [ ]:
#랜덤포레스트모델의 정확도
from sklearn.ensemble import RandomForestClassifier   

rfc = RandomForestClassifier(
                            n_estimators=150, 
                            max_depth=None, 
                            min_samples_split=2, 
                            #  class_weight='balanced', 
                            min_samples_leaf=2,
                            )
rfc.fit(x_train, y_train)
rfc.score(x_val, y_val)

0.8614395368259062

In [ ]:
# # 나이브 베이즈모델의 정확도
# from sklearn.naive_bayes import GaussianNB

# gnb = GaussianNB()
# gnb.fit(xx_train, yy_train)
# gnb.score(xx_val, yy_val)

In [ ]:
# # GB 모델의 정확도
# gb = GradientBoostingClassifier()
# gb.fit(xx_train, yy_train)
# gb.score(xx_val, yy_val)

In [ ]:
# #서포트벡터머신 모델의 정확도
# from sklearn import svm 

# svmm = svm.SVC()
# svmm.fit(xx_train, yy_train)
# svmm.score(xx_train, yy_train)